In [ ]:
# from pytubefix import YouTube

# urls = 'https://www.youtube.com/watch?v=H2dEZftiAIk'

# entry = YouTube(urls).title

# vid = YouTube(urls)
# audio_download = vid.streams.get_audio_only()
# audio_download.download(mp3=True, filename="H2dEZftiAIk", output_path="/tmp")

In [ ]:
# import youtube_dl

# ydl_opts = {
#     'format': 'worstaudio/worst',
#     'postprocessors': [{
#         'key': 'FFmpegExtractAudio',
#         'preferredcodec': 'mp3',
#         'preferredquality': '192',
#     }],
# }
# with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#     ydl.download(['http://www.youtube.com/watch?v=C0DPdy98e4c'])

In [1]:
from utils import download_youtube_audio, transcribe

audio_file = download_youtube_audio('2htwBFFgNXg')

print(audio_file)

if audio_file is not None:
  transcription = transcribe(audio_file)

transcription

/Users/josephcheng/Projects/yue-subtitle-api/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/var/folders/wp/0zldm9wx1m54950_qnk812f80000gn/T/2htwBFFgNXg.mp3


[src/libmpg123/parse.c:skip_junk():1276] error: Giving up searching valid MPEG header after 65536 bytes of junk.
/Users/josephcheng/Projects/yue-subtitle-api/utils.py:270: UserWarning: PySoundFile failed. Trying audioread instead.
  speech = librosa.load(audio_file, sr=16000)[0]
/Users/josephcheng/Projects/yue-subtitle-api/.venv/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


[TranscribeResult(text=各位今日, start_time=0.6566071428571428, end_time=1.3587500000000001),
 TranscribeResult(text=我要講嘅成語就係叫做屠龍之極啦嗯, start_time=1.3587500000000001, end_time=4.459880952380952),
 TranscribeResult(text=冇錯啦, start_time=4.635416666666666, end_time=5.3375595238095235),
 TranscribeResult(text=從前有個叫朱平萬嘅青年好多人都覺得, start_time=5.747142857142856, end_time=8.49720238095238),
 TranscribeResult(text=佢, start_time=8.555714285714286, end_time=8.672738095238095),
 TranscribeResult(text=係個怪人嚟, start_time=8.672738095238095, end_time=9.96),
 TranscribeResult(text=嘿, start_time=10.964347826086957, end_time=11.64),
 TranscribeResult(text=佢哎啊, start_time=14.93878787878788, end_time=17.91),
 TranscribeResult(text=嗯, start_time=19.63235294117647, end_time=20.04),
 TranscribeResult(text=嘿, start_time=22.151176470588233, end_time=22.459999999999997),
 TranscribeResult(text=誒, start_time=22.985384615384614, end_time=23.33),
 TranscribeResult(text=嘿, start_time=24.83285714285714, end_time=25.09),
 Tra

In [17]:
from pysrt import SubRipFile
from pysrt import SubRipItem
from pysrt import SubRipTime

out = SubRipFile()

for i, t in enumerate(transcription):
  start = SubRipTime(seconds=t.start_time)
  end = SubRipTime(seconds=t.end_time)
  item = SubRipItem(index=i, start=start, end=end, text=t.text)
  out.append(item)

out.save('subtitles.srt')

In [25]:
model = bert_model.model
tokenizer = bert_model.tokenizer

text = "龍[MASK]係傳說中嘅動物豬皮萬噹然等唔到龍出嚟"
model_inputs = tokenizer(text, return_tensors="pt")
test1_label = "隻"
test2_label = "只"
test1_index = tokenizer.convert_tokens_to_ids(test1_label)
test2_index = tokenizer.convert_tokens_to_ids(test2_label)
mask_index = model_inputs.input_ids[0].tolist().index(tokenizer.mask_token_id)
vocab_size = len(tokenizer.get_vocab())
inputs_onnx = {k: v.cpu().detach().numpy()
                for k, v in model_inputs.items()}
predictions = model.run(None, inputs_onnx)
lm_logits = predictions[0]

print(test1_label, lm_logits[0, mask_index, test1_index])
print(test2_label, lm_logits[0, mask_index, test2_index])

隻 6.985522
只 7.0312243
